In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("../data/data.db")

In [ ]:
papers_df = pd.read_sql_query("SELECT * FROM papers", conn)
papers_df = papers_df[["doi", "reference_count", "is_referenced_count", "created_date", "title"]]
papers_df.head()

In [ ]:
edges_query = """
WITH RECURSIVE citation_chain AS (
    SELECT src_doi, dest_doi, 1 AS depth
    FROM paper_references
    WHERE src_doi = ?

    UNION ALL

    SELECT pr.src_doi, pr.dest_doi, cc.depth + 1
    FROM paper_references pr
    INNER JOIN citation_chain cc ON pr.src_doi = cc.dest_doi
    WHERE cc.depth < 5
)
SELECT * FROM citation_chain;
"""

edges_df = pd.read_sql_query(edges_query, conn, params=["10.1103/revmodphys.81.109"])
edges_df.head()

,src_doi,dest_doi,depth
0,10.1103/revmodphys.81.109,10.1007/978-3-7643-7978-0,1
1,10.1103/revmodphys.81.109,10.1007/978-94-011-4038-6,1
2,10.1103/revmodphys.81.109,10.1007/s002570050384,1
3,10.1103/revmodphys.81.109,10.1007/s10909-006-9286-5,1
4,10.1103/revmodphys.81.109,10.1007/s12034-008-0090-5,1


In [ ]:
nodes_query = """
WITH RECURSIVE citation_chain AS (
    SELECT src_doi, dest_doi, 1 AS depth
    FROM paper_references
    WHERE src_doi = ?

    UNION ALL

    SELECT pr.src_doi, pr.dest_doi, cc.depth + 1
    FROM paper_references pr
    INNER JOIN citation_chain cc ON pr.src_doi = cc.dest_doi
    WHERE cc.depth < 5
)
SELECT * FROM papers 
WHERE doi IN (
    SELECT src_doi FROM citation_chain
    UNION
    SELECT dest_doi FROM citation_chain
)
"""

nodes_df = pd.read_sql_query(nodes_query, conn, params=["10.1103/revmodphys.81.109"])
nodes_df.head()

,doi,reference_count,is_referenced_count,publisher,created_date,type,title,url
0,10.1002/adma.200701059,31,267,Wiley,2007-10-05T09:22:59Z,journal-article,Printed Graphene Circuits,https://doi.org/10.1002/adma.200701059
1,10.1016/j.ssc.2007.04.023,45,786,Elsevier BV,2007-04-30T10:49:32Z,journal-article,Epitaxial graphene,https://doi.org/10.1016/j.ssc.2007.04.023
2,10.1016/j.ssc.2007.04.024,29,71,Elsevier BV,2007-04-30T14:49:32Z,journal-article,Charge and spin transport at the quantum Hall ...,https://doi.org/10.1016/j.ssc.2007.04.024
3,10.1016/j.ssc.2007.04.035,32,101,Elsevier BV,2007-04-30T11:13:15Z,journal-article,Graphene: A pseudochiral Fermi liquid,https://doi.org/10.1016/j.ssc.2007.04.035
4,10.1016/j.ssc.2007.06.035,29,25,Elsevier BV,2007-07-19T07:18:37Z,journal-article,Interplay between lattice-scale physics and th...,https://doi.org/10.1016/j.ssc.2007.06.035
